In [56]:
# Data Handling and Processing
import pandas as pd
import polars as pl
import numpy as np
from datetime import timedelta
import calendar
from dateutil.relativedelta import relativedelta
import random
import json

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors

# Machine Learning and Data Preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.base import clone
from sklearn.model_selection import ParameterGrid

# Utilities and Miscellaneous
from joblib import dump, load
import os
import re


In [2]:
pl.Config.set_tbl_width_chars(width= 200)
pl.Config.set_tbl_cols(12)
pl.Config.set_tbl_rows(10)

polars.config.Config

## Data Preparation For Modeling

In [3]:
train_df = pl.read_csv('../data/input/train.csv', try_parse_dates=True, dtypes={'store_nbr': str})
print(train_df)

shape: (3_000_888, 6)
┌─────────┬────────────┬───────────┬────────────────────────────┬──────────┬─────────────┐
│ id      ┆ date       ┆ store_nbr ┆ family                     ┆ sales    ┆ onpromotion │
│ ---     ┆ ---        ┆ ---       ┆ ---                        ┆ ---      ┆ ---         │
│ i64     ┆ date       ┆ str       ┆ str                        ┆ f64      ┆ i64         │
╞═════════╪════════════╪═══════════╪════════════════════════════╪══════════╪═════════════╡
│ 0       ┆ 2013-01-01 ┆ 1         ┆ AUTOMOTIVE                 ┆ 0.0      ┆ 0           │
│ 1       ┆ 2013-01-01 ┆ 1         ┆ BABY CARE                  ┆ 0.0      ┆ 0           │
│ 2       ┆ 2013-01-01 ┆ 1         ┆ BEAUTY                     ┆ 0.0      ┆ 0           │
│ 3       ┆ 2013-01-01 ┆ 1         ┆ BEVERAGES                  ┆ 0.0      ┆ 0           │
│ 4       ┆ 2013-01-01 ┆ 1         ┆ BOOKS                      ┆ 0.0      ┆ 0           │
│ …       ┆ …          ┆ …         ┆ …                          ┆ … 

In [4]:
q= pl.scan_csv('../data/input/test.csv', try_parse_dates=True, dtypes={'store_nbr': str})
test_df = q.collect()
print(test_df)

shape: (28_512, 5)
┌─────────┬────────────┬───────────┬────────────────────────────┬─────────────┐
│ id      ┆ date       ┆ store_nbr ┆ family                     ┆ onpromotion │
│ ---     ┆ ---        ┆ ---       ┆ ---                        ┆ ---         │
│ i64     ┆ date       ┆ str       ┆ str                        ┆ i64         │
╞═════════╪════════════╪═══════════╪════════════════════════════╪═════════════╡
│ 3000888 ┆ 2017-08-16 ┆ 1         ┆ AUTOMOTIVE                 ┆ 0           │
│ 3000889 ┆ 2017-08-16 ┆ 1         ┆ BABY CARE                  ┆ 0           │
│ 3000890 ┆ 2017-08-16 ┆ 1         ┆ BEAUTY                     ┆ 2           │
│ 3000891 ┆ 2017-08-16 ┆ 1         ┆ BEVERAGES                  ┆ 20          │
│ 3000892 ┆ 2017-08-16 ┆ 1         ┆ BOOKS                      ┆ 0           │
│ …       ┆ …          ┆ …         ┆ …                          ┆ …           │
│ 3029395 ┆ 2017-08-31 ┆ 9         ┆ POULTRY                    ┆ 1           │
│ 3029396 ┆ 2017-08-3

In [5]:
# Step 1: Concatenate train and test dataframes vertically
combined_df = pl.concat([train_df, test_df], how='diagonal')


# Step 2: Create a new column "label" to distinguish between train and test samples
combined_df = combined_df.with_columns(
    label=pl.when(pl.col('id')<=3000887)
    .then(pl.lit('train'))
    .otherwise(pl.lit('test'))
)


In [6]:
# Map day of the week integers to day names
day_names = list(calendar.day_name)

day_name_dict = {key+1:value for key, value in enumerate(day_names)}


In [7]:
combined_df = combined_df.with_columns(
    dayofweek=pl.col('date').dt.weekday().cast(pl.Int8)
    .replace(day_name_dict, default=None)
)

weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekend = ['Saturday', 'Sunday']

combined_df = combined_df.with_columns(
    working_day=pl.when(pl.col('dayofweek').is_in(weekday))
    .then(pl.lit('Yes'))
    .otherwise(pl.lit('No'))
)

print(combined_df)


shape: (3_029_400, 9)
┌─────────┬────────────┬───────────┬────────────────────────────┬───────┬─────────────┬───────┬───────────┬─────────────┐
│ id      ┆ date       ┆ store_nbr ┆ family                     ┆ sales ┆ onpromotion ┆ label ┆ dayofweek ┆ working_day │
│ ---     ┆ ---        ┆ ---       ┆ ---                        ┆ ---   ┆ ---         ┆ ---   ┆ ---       ┆ ---         │
│ i64     ┆ date       ┆ str       ┆ str                        ┆ f64   ┆ i64         ┆ str   ┆ str       ┆ str         │
╞═════════╪════════════╪═══════════╪════════════════════════════╪═══════╪═════════════╪═══════╪═══════════╪═════════════╡
│ 0       ┆ 2013-01-01 ┆ 1         ┆ AUTOMOTIVE                 ┆ 0.0   ┆ 0           ┆ train ┆ Tuesday   ┆ Yes         │
│ 1       ┆ 2013-01-01 ┆ 1         ┆ BABY CARE                  ┆ 0.0   ┆ 0           ┆ train ┆ Tuesday   ┆ Yes         │
│ 2       ┆ 2013-01-01 ┆ 1         ┆ BEAUTY                     ┆ 0.0   ┆ 0           ┆ train ┆ Tuesday   ┆ Yes         │
│ 

### Adding Oil Price Data

In [8]:

oil_df = pl.read_csv('../data/input/oil.csv')
oil_df = oil_df.with_columns(pl.col('date').str.strptime(pl.Date, '%m/%d/%y'))
oil_df = oil_df.rename({'dcoilwtico': 'oil_price'})
print(oil_df)

shape: (1_218, 2)
┌────────────┬───────────┐
│ date       ┆ oil_price │
│ ---        ┆ ---       │
│ date       ┆ f64       │
╞════════════╪═══════════╡
│ 2013-01-01 ┆ null      │
│ 2013-01-02 ┆ 93.14     │
│ 2013-01-03 ┆ 92.97     │
│ 2013-01-04 ┆ 93.12     │
│ 2013-01-07 ┆ 93.2      │
│ …          ┆ …         │
│ 2017-08-25 ┆ 47.65     │
│ 2017-08-28 ┆ 46.4      │
│ 2017-08-29 ┆ 46.46     │
│ 2017-08-30 ┆ 45.96     │
│ 2017-08-31 ┆ 47.26     │
└────────────┴───────────┘


In [9]:
combined_df = combined_df.join(oil_df, how='left', on='date', validate='m:1')
print(combined_df)

shape: (3_029_400, 10)
┌─────────┬────────────┬───────────┬────────────────────────────┬───────┬─────────────┬───────┬───────────┬─────────────┬───────────┐
│ id      ┆ date       ┆ store_nbr ┆ family                     ┆ sales ┆ onpromotion ┆ label ┆ dayofweek ┆ working_day ┆ oil_price │
│ ---     ┆ ---        ┆ ---       ┆ ---                        ┆ ---   ┆ ---         ┆ ---   ┆ ---       ┆ ---         ┆ ---       │
│ i64     ┆ date       ┆ str       ┆ str                        ┆ f64   ┆ i64         ┆ str   ┆ str       ┆ str         ┆ f64       │
╞═════════╪════════════╪═══════════╪════════════════════════════╪═══════╪═════════════╪═══════╪═══════════╪═════════════╪═══════════╡
│ 0       ┆ 2013-01-01 ┆ 1         ┆ AUTOMOTIVE                 ┆ 0.0   ┆ 0           ┆ train ┆ Tuesday   ┆ Yes         ┆ null      │
│ 1       ┆ 2013-01-01 ┆ 1         ┆ BABY CARE                  ┆ 0.0   ┆ 0           ┆ train ┆ Tuesday   ┆ Yes         ┆ null      │
│ 2       ┆ 2013-01-01 ┆ 1         ┆ BE

In [10]:
combined_df =combined_df.with_columns(pl.col('oil_price').fill_null(strategy='forward')
                            .fill_null(strategy='backward')
)

### Importing stores data

In [11]:
stores_df = pl.read_csv('../data/input/stores.csv',  dtypes={'store_nbr': str, 'cluster': str})
print(stores_df)

shape: (54, 5)
┌───────────┬───────────────┬────────────────────────────────┬──────┬─────────┐
│ store_nbr ┆ city          ┆ state                          ┆ type ┆ cluster │
│ ---       ┆ ---           ┆ ---                            ┆ ---  ┆ ---     │
│ str       ┆ str           ┆ str                            ┆ str  ┆ str     │
╞═══════════╪═══════════════╪════════════════════════════════╪══════╪═════════╡
│ 1         ┆ Quito         ┆ Pichincha                      ┆ D    ┆ 13      │
│ 2         ┆ Quito         ┆ Pichincha                      ┆ D    ┆ 13      │
│ 3         ┆ Quito         ┆ Pichincha                      ┆ D    ┆ 8       │
│ 4         ┆ Quito         ┆ Pichincha                      ┆ D    ┆ 9       │
│ 5         ┆ Santo Domingo ┆ Santo Domingo de los Tsachilas ┆ D    ┆ 4       │
│ …         ┆ …             ┆ …                              ┆ …    ┆ …       │
│ 50        ┆ Ambato        ┆ Tungurahua                     ┆ A    ┆ 14      │
│ 51        ┆ Guayaquil  

In [12]:
combined_df = combined_df.join(stores_df, how='left', on='store_nbr', validate='m:1')
print(combined_df)

shape: (3_029_400, 14)
┌─────────┬────────────┬───────────┬────────────────────────────┬───────┬─────────────┬───┬─────────────┬───────────┬───────┬───────────┬──────┬─────────┐
│ id      ┆ date       ┆ store_nbr ┆ family                     ┆ sales ┆ onpromotion ┆ … ┆ working_day ┆ oil_price ┆ city  ┆ state     ┆ type ┆ cluster │
│ ---     ┆ ---        ┆ ---       ┆ ---                        ┆ ---   ┆ ---         ┆   ┆ ---         ┆ ---       ┆ ---   ┆ ---       ┆ ---  ┆ ---     │
│ i64     ┆ date       ┆ str       ┆ str                        ┆ f64   ┆ i64         ┆   ┆ str         ┆ f64       ┆ str   ┆ str       ┆ str  ┆ str     │
╞═════════╪════════════╪═══════════╪════════════════════════════╪═══════╪═════════════╪═══╪═════════════╪═══════════╪═══════╪═══════════╪══════╪═════════╡
│ 0       ┆ 2013-01-01 ┆ 1         ┆ AUTOMOTIVE                 ┆ 0.0   ┆ 0           ┆ … ┆ Yes         ┆ 93.14     ┆ Quito ┆ Pichincha ┆ D    ┆ 13      │
│ 1       ┆ 2013-01-01 ┆ 1         ┆ BABY CARE 

### Importing Holiday-Events Data

In [13]:
holiday_events_df = pl.read_csv('../data/input/holidays_events_mod_pl.csv', try_parse_dates=True)
print(holiday_events_df)

shape: (350, 10)
┌────────────┬────────────┬──────────┬─────────────┬───────────────────────────────┬─────────────┬───────────┬─────────────┬─────────────────────┬─────────────────────┐
│ date       ┆ type       ┆ locale   ┆ locale_name ┆ description                   ┆ transferred ┆ dayofweek ┆ working_day ┆ description_base_es ┆ description_base_en │
│ ---        ┆ ---        ┆ ---      ┆ ---         ┆ ---                           ┆ ---         ┆ ---       ┆ ---         ┆ ---                 ┆ ---                 │
│ date       ┆ str        ┆ str      ┆ str         ┆ str                           ┆ bool        ┆ str       ┆ str         ┆ str                 ┆ str                 │
╞════════════╪════════════╪══════════╪═════════════╪═══════════════════════════════╪═════════════╪═══════════╪═════════════╪═════════════════════╪═════════════════════╡
│ 2012-03-02 ┆ Holiday    ┆ Local    ┆ Manta       ┆ Fundacion de Manta            ┆ false       ┆ Friday    ┆ No          ┆ Fundacion    

In [14]:
type1_unique = holiday_events_df.filter(pl.col('locale')=='National').unique(subset=['date'], maintain_order=True)
type2_unique = holiday_events_df.filter(pl.col('locale')!='National').unique(subset=['date', 'locale_name'], maintain_order=True)

# Combine both types of duplicates into a single boolean Series
holiday_events_df = pl.concat([type1_unique, type2_unique])

print(holiday_events_df)

shape: (343, 10)
┌────────────┬────────────┬──────────┬─────────────┬───────────────────────────────────┬─────────────┬───────────┬─────────────┬─────────────────────────────┬────────────────────────┐
│ date       ┆ type       ┆ locale   ┆ locale_name ┆ description                       ┆ transferred ┆ dayofweek ┆ working_day ┆ description_base_es         ┆ description_base_en    │
│ ---        ┆ ---        ┆ ---      ┆ ---         ┆ ---                               ┆ ---         ┆ ---       ┆ ---         ┆ ---                         ┆ ---                    │
│ date       ┆ str        ┆ str      ┆ str         ┆ str                               ┆ bool        ┆ str       ┆ str         ┆ str                         ┆ str                    │
╞════════════╪════════════╪══════════╪═════════════╪═══════════════════════════════════╪═════════════╪═══════════╪═════════════╪═════════════════════════════╪════════════════════════╡
│ 2012-08-10 ┆ Holiday    ┆ National ┆ Ecuador     ┆ Primer Gri

In [15]:
# First condition (when locale=='National')
combined_df = combined_df.join(holiday_events_df.filter(pl.col('locale')=='National'), how='left', left_on='date', right_on='date', suffix='_national')

In [16]:
# Second condition (when locale!='National')
combined_df = combined_df.join(holiday_events_df.filter(pl.col('locale')!='National'), how='left', left_on=['date', 'state'], right_on=['date', 'locale_name'], suffix='_state')

In [17]:
train_df = []
test_df = []
oil_df= []
stores_df = []
# holiday_events_df = []

#### Feature Engineering

In [18]:
combined_df = combined_df.with_columns(
    working_day_final=pl.when((pl.col('working_day_national').is_not_null()) & (pl.col('working_day_state').is_null()))
    .then(pl.col('working_day_national'))
    .when((pl.col('working_day_national').is_null()) & (pl.col('working_day_state').is_not_null()))
    .then(pl.col('working_day_state'))
    .when((pl.col('working_day_national').is_not_null()) & (pl.col('working_day_state').is_not_null()) & (pl.col('state')==pl.col('locale_name')))
    .then(pl.col('working_day_state'))
    .when((pl.col('working_day_national').is_not_null()) & (pl.col('working_day_state').is_not_null()) & (pl.col('state')!=pl.col('locale_name')))
    .then(pl.col('working_day_national'))
    .otherwise(pl.col('working_day'))
)

combined_df = combined_df.with_columns(
    type_of_day=pl.when((pl.col('type_national').is_not_null()) & (pl.col('type_state').is_null()))
    .then(pl.col('type_national'))
    .when((pl.col('type_national').is_null()) & (pl.col('type_state').is_not_null()))
    .then(pl.col('type_state'))
    .when((pl.col('type_national').is_not_null()) & (pl.col('type_state').is_not_null()) & (pl.col('state')==pl.col('locale_name')))
    .then(pl.col('type_state'))
    .when((pl.col('type_national').is_not_null()) & (pl.col('type_state').is_not_null()) & (pl.col('state')!=pl.col('locale_name')))
    .then(pl.col('type_national'))
    .otherwise(pl.lit('regular_day'))
)

combined_df = combined_df.with_columns(
    description_final=pl.when((pl.col('description_base_en').is_not_null()) & (pl.col('description_base_en_state').is_null()))
    .then(pl.col('description_base_en'))
    .when((pl.col('description_base_en').is_null()) & (pl.col('description_base_en_state').is_not_null()))
    .then(pl.col('description_base_en_state'))
    .when((pl.col('description_base_en').is_not_null()) & (pl.col('description_base_en_state').is_not_null()) & (pl.col('state')==pl.col('locale_name')))
    .then(pl.col('description_base_en_state'))
    .when((pl.col('description_base_en').is_not_null()) & (pl.col('description_base_en_state').is_not_null()) & (pl.col('state')!=pl.col('locale_name')))
    .then(pl.col('description_base_en'))
    .otherwise(pl.lit('regular_day'))
)

combined_df = combined_df.with_columns(
    locale_name_final=pl.when((pl.col('locale_name').is_not_null()) & (pl.col('locale_state').is_null()))
    .then(pl.lit('national'))
    .when((pl.col('locale_name').is_null()) & (pl.col('locale_state').is_not_null()))
    .then(pl.lit('state'))
    .when((pl.col('locale_name').is_not_null()) & (pl.col('locale_state').is_not_null()))
    .then(pl.lit('national+state'))
    .otherwise(pl.lit('state'))
)



In [19]:
combined_df = combined_df.with_columns(
    type_weekday=pl.when(pl.col('dayofweek').is_in(weekday))
    .then(pl.lit('weekday'))
    .otherwise(pl.lit('weekend'))
)

In [20]:
combined_df = combined_df.with_columns(
    # is_salary_day=pl.when((pl.col('date').dt.day().cast(pl.Int32) == pl.lit(15).cast(pl.Int32)) | (pl.col('date').dt.day() == pl.col('date').dt.month_end().dt.day()))
    is_salary_day=pl.when((pl.col('date').dt.day() == 15) | (pl.col('date').dt.day() == pl.col('date').dt.month_end().dt.day()))
    .then(pl.lit('Yes'))
    .otherwise(pl.lit('No'))
)

In [21]:
combined_df = combined_df.with_columns(
    month=pl.col('date').dt.strftime("%B"),
    year=pl.col('date').dt.strftime("%Y"),
    day=pl.col('date').dt.strftime("%d")
)

In [22]:

random_number = random.randint(0, 3e6)

print(combined_df.filter(
    pl.col('id')==random_number
    ).transpose(include_header = True)
)

shape: (40, 2)
┌───────────────┬─────────────┐
│ column        ┆ column_0    │
│ ---           ┆ ---         │
│ str           ┆ str         │
╞═══════════════╪═════════════╡
│ id            ┆ 244635      │
│ date          ┆ 2013-05-18  │
│ store_nbr     ┆ 23          │
│ family        ┆ CELEBRATION │
│ sales         ┆ 0.0         │
│ …             ┆ …           │
│ type_weekday  ┆ weekend     │
│ is_salary_day ┆ No          │
│ month         ┆ May         │
│ year          ┆ 2013        │
│ day           ┆ 18          │
└───────────────┴─────────────┘


### Modeling

In [23]:
features =  { # flag to drop, type of variable
'id'	:	[	'Yes'	,	'id'	]	,
'date'	:	[	'No'	,	'date'	]	,
'store_nbr'	:	[	'No'	,	'store_nbr'	]	,
'family'	:	[	'No'	,	'family'	]	,
'sales'	:	[	'No'	,	'target'	]	,
'onpromotion'	:	[	'No'	,	'numerical'	]	,
'label'	:	[	'No'	,      	'label'	]	,
'dayofweek'	:	[	'No'	,	'categorical'	]	,
'working_day'	:	[	'Yes'	,	'categorical'	]	,
'oil_price'	:	[	'No'	,	'numerical'	]	,
'city'	:	[	'Yes'	,	'categorical'	]	,
'state'	:	[	'Yes'	,	'categorical'	]	,
'type'	:	[	'No'	,	'categorical'	]	,
'cluster'	:	[	'No'	,	'categorical'	]	,
'type_national'	:	[	'Yes'	,	'categorical'	]	,
'locale'	:	[	'Yes'	,	'categorical'	]	,
'locale_name'	:	[	'Yes'	,	'categorical'	]	,
'description'	:	[	'Yes'	,	'categorical'	]	,
'transferred'	:	[	'Yes'	,	'categorical'	]	,
'dayofweek_national'	:	[	'Yes'	,	'categorical'	]	,
'working_day_national'	:	[	'Yes'	,	'categorical'	]	,
'description_base_es'	:	[	'Yes'	,	'categorical'	]	,
'description_base_en'	:	[	'Yes'	,	'categorical'	]	,
'type_state'	:	[	'Yes'	,	'categorical'	]	,
'locale_state'	:	[	'Yes'	,	'categorical'	]	,
# 'locale_name_state'	:	[	'Yes'	,	'categorical'	]	,       # this column does not exist in polars version of combined_df
'description_state'	:	[	'Yes'	,	'categorical'	]	,
'transferred_state'	:	[	'Yes'	,	'categorical'	]	,
'dayofweek_state'	:	[	'Yes'	,	'categorical'	]	,
'working_day_state'	:	[	'Yes'	,	'categorical'	]	,
'description_base_es_state'	:	[	'Yes'	,	'categorical'	]	,
'description_base_en_state'	:	[	'Yes'	,	'categorical'	]	,
'working_day_final'	:	[	'No'	,	'categorical'	]	,
'type_of_day'	:	[	'No'	,	'categorical'	]	,
'description_final'	:	[	'No'	,	'categorical'	]	,
'locale_name_final'	:	[	'No'	,	'categorical'	]	,
'type_weekday'	:	[	'No'	,	'categorical'	]	,
'is_salary_day'	:	[	'No'	,	'categorical'	]	,
'month'	:	[	'No'	,	'categorical'	]	,
'year' 	:	[	'No'	,	'categorical'	]	,
'day'	:	[	'No'	,	'categorical'	]	
}

In [24]:
def get_columns(features):
    """Get columns to drop, numerical columns, and categorical columns based on feature dictionary."""
    drop_cols = [col for col, val in features.items() if val[0] == 'Yes']
    num_cols = [col for col, val in features.items() if val[1] == 'numerical' and val[0] == 'No']
    cat_cols = [col for col, val in features.items() if val[1] == 'categorical' and val[0] == 'No']
    return drop_cols, num_cols, cat_cols

# Get columns based on feature dictionary
drop_cols, num_cols, cat_cols = get_columns(features)



In [25]:
combined_df = combined_df.drop(drop_cols)


In [26]:
combined_df = combined_df.to_pandas()
combined_df = combined_df.sort_values(by='date')


In [27]:
combined_df.columns

Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'label',
       'dayofweek', 'oil_price', 'type', 'cluster', 'working_day_final',
       'type_of_day', 'description_final', 'locale_name_final', 'type_weekday',
       'is_salary_day', 'month', 'year', 'day'],
      dtype='object')

In [28]:
# Separate the DataFrame into training and test sets based on the 'label' column
train_df = combined_df[combined_df['label'] == 'train']
test_df = combined_df[combined_df['label'] == 'test']

In [29]:
def get_zero_sales_families(df):
    recent_date = df['date'].max() - timedelta(days=15)
    zero_sales = df[df['date'] > recent_date].groupby('family')['sales'].sum() == 0
    return zero_sales[zero_sales].index.tolist()

zero_sales_families = get_zero_sales_families(train_df)

In [31]:
# Identify the categorical and numerical features
# categorical_features = ['store_nbr', 'dayofweek', 'type', 'cluster', 
#                         'working_day_final', 'type_of_day', 'description_final', 
#                         'locale_name_final', 'type_weekday', 'is_salary_day', 'month', 'year', 'day']
# numerical_features = ['onpromotion', 'oil_price']

# Define the preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                            ('scaler', StandardScaler())]), num_cols),
    ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                            ('encoder', OneHotEncoder(handle_unknown='ignore'))]), cat_cols),
])

# Model definition
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(objective='reg:squaredlogerror', n_estimators=100, random_state=42))
])

In [32]:
def get_columns(features):
    """Get columns to drop, numerical columns, and categorical columns based on feature dictionary."""
    drop_cols = [col for col, val in features.items() if val[0] == 'Yes']
    num_cols = [col for col, val in features.items() if val[1] == 'numerical' and val[0] == 'No']
    cat_cols = [col for col, val in features.items() if val[1] == 'categorical' and val[0] == 'No']
    return drop_cols, num_cols, cat_cols

# Get columns based on feature dictionary
drop_cols, num_cols, cat_cols = get_columns(features)



In [33]:
def rmsle(y_true, y_pred):
    """
    Compute the Root Mean Squared Log Error for two arrays.
    """
    assert len(y_true) == len(y_pred), "Mismatched length between true and predicted values"
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

In [34]:
def enhanced_time_series_cv_rmsle(model, X, y, dates, n_splits):
    """
    Perform time series cross-validation, ensuring no leakage between training and test sets.
    
    :param model: The model to be trained and validated.
    :param X: Features DataFrame.
    :param y: Target variable Series.
    :param dates: Series or array containing the date for each observation in X.
    :param n_splits: Number of splits for cross-validation.
    :return: List of RMSLE scores for each fold.
    """
    rmsle_scores = []
    unique_dates = np.array(sorted(dates.unique()))
    split_indices = np.array_split(unique_dates, n_splits + 1)
    
    for i in range(1, len(split_indices)):
        train_dates = np.concatenate(split_indices[:i])
        test_dates = split_indices[i]
        
        train_indices = dates[dates.isin(train_dates)].index
        test_indices = dates[dates.isin(test_dates)].index
        
        X_train, X_test = X.loc[train_indices], X.loc[test_indices]
        y_train, y_test = y.loc[train_indices], y.loc[test_indices]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        predictions = np.clip(predictions, a_min=0, a_max=None)  # Ensure predictions are non-negative
        rmsle_score = rmsle(y_test, predictions)
        rmsle_scores.append(rmsle_score)
    
    return rmsle_scores


In [37]:
def sanitize_filename(filename):
    # Replace any character that is not alphanumeric or an underscore with an underscore
    sanitized = re.sub(r'[^a-zA-Z0-9_]', '_', filename)
    return sanitized

In [35]:
hyperparameter_grid = {
    'model__learning_rate': [0.01, 0.1, 0.3],
    'model__n_estimators': [100, 500, 1000],
    'model__max_depth': [3, 6, 9],
    'model__subsample': [0.7, 0.8, 1.0],
    'model__colsample_bytree': [0.7, 0.8, 1.0],
}


In [38]:

best_scores = []
family_models = {}


model_directory = '../models/'  # Define the directory to store models
os.makedirs(model_directory, exist_ok=True)  # Create the directory if it doesn't exist

for family in train_df['family'].unique():
    # Replace slashes in the model_identifier to prevent directory path issues
    model_identifier = sanitize_filename(f'{family}')
    model_path = os.path.join(model_directory, f'{model_identifier}_model.joblib')

    
    # Check if the model already exists
    if os.path.exists(model_path):
        print(f"Model for {family} already exists. Skipping training.")
        family_model = load(model_path)  # Load the existing model
    else:
        print(f"Training model for {family}.")
        family_df = train_df[train_df['family'] == family]
        
        X_family = family_df.drop(['sales', 'label', 'date', 'family'], axis=1)
        y_family = family_df['sales']
        dates_family = family_df['date']
        
        # Define and fit the model pipeline (including hyperparameter tuning if applicable)
        family_model_pipeline = clone(model_pipeline)
        
        best_score = np.inf
        best_params = None
        
        for params in ParameterGrid(hyperparameter_grid):
            family_model_pipeline.set_params(**params)
            avg_rmsle = np.mean(enhanced_time_series_cv_rmsle(family_model_pipeline, X_family, y_family, dates_family, n_splits=3))
            
            if avg_rmsle < best_score:
                best_score = avg_rmsle
                best_params = params
        
        print(f"Best RMSLE for {family}: {round(best_score, 2)} with params {best_params}")
        
        # Retrain the model on the entire dataset with the best parameters
        family_model_pipeline.set_params(**best_params)
        family_model_pipeline.fit(X_family, y_family)
        
        # Save the trained model
        dump(family_model_pipeline, model_path)
        
        family_models[family] = family_model_pipeline

Model for AUTOMOTIVE already exists. Skipping training.
Model for CELEBRATION already exists. Skipping training.
Training model for BREAD/BAKERY.
Best RMSLE for BREAD/BAKERY: 1.43 with params {'model__colsample_bytree': 1.0, 'model__learning_rate': 0.3, 'model__max_depth': 9, 'model__n_estimators': 1000, 'model__subsample': 0.7}
Training model for BOOKS.
Best RMSLE for BOOKS: 0.14 with params {'model__colsample_bytree': 0.7, 'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 0.7}
Training model for BEVERAGES.
Best RMSLE for BEVERAGES: 1.86 with params {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.3, 'model__max_depth': 6, 'model__n_estimators': 1000, 'model__subsample': 1.0}
Training model for BEAUTY.
Best RMSLE for BEAUTY: 0.65 with params {'model__colsample_bytree': 1.0, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 1000, 'model__subsample': 1.0}
Training model for BABY CARE.
Best RMSLE for BA

In [51]:
def get_zero_sales_families(df, cutoff_date):
    recent_df = df[df['date'] > cutoff_date]
    zero_sales_families = recent_df.groupby('family')['sales'].sum()
    return zero_sales_families[zero_sales_families == 0].index.tolist()



In [52]:
max_date_train = train_df['date'].max()
cutoff_date = max_date_train - pd.Timedelta(days=21)
zero_sales_families = get_zero_sales_families(train_df, cutoff_date)


In [55]:
# Assuming zero_sales_families is a list of families with zero sales in the last period
submission = pd.DataFrame()

for family in test_df['family'].unique():
    # Copy the filtered test_df for the specific family
    # and reset the index while keeping the index (which serves as 'id') in a column
    family_test_df = test_df[test_df['family'] == family].copy().reset_index()
    family_test_df.rename(columns={'index': 'id'}, inplace=True)

    # If this family should have zero sales, set predictions directly
    if family in zero_sales_families:
        family_test_df['sales'] = 0
    else:
        model_path = os.path.join(model_directory, f'{sanitize_filename(family)}_model.joblib')
        
        if os.path.exists(model_path):
            family_model = load(model_path)
        else:
            print(f"No pre-trained model found for {family}. Skipping.")
            continue  # Skip this family if no model is found
        
        X_test_family = family_test_df.drop(['sales', 'label', 'date', 'family', 'id'], axis=1)
        
        # Make predictions for families not in zero_sales_families
        predictions = family_model.predict(X_test_family)
        predictions = np.clip(predictions, a_min=0, a_max=None)  # Ensure non-negative predictions
        family_test_df['sales'] = predictions
    
    # Append the predictions for the current family to the submission DataFrame
    submission = pd.concat([submission, family_test_df[['id', 'sales']]], ignore_index=True)

# Ensure 'id' is an integer for the submission file
submission['id'] = submission['id'].astype(int)
submission.to_csv('final_sales_predictions_family.csv', index=False)
